In [1]:
from show_graph import draw_dot
from engine import Value
from nn import MLP
import numpy as np

In [2]:
x = [
    [2.0,3.0,-1.0],
    [3.0,-1.0,0.5],
    [0.5,1.0,1.0],
    [1.0,1.0,-1.0]
     ] 
y = [1,-1,-1,1]
n = MLP(3,[4, 4, 1])
for i in range(20):
    y_pred = [n(i) for i in x]
    # print(y_pred)
    loss = sum((y_pred1-y1)**2 for y_pred1, y1 in zip(y_pred,y))
    print(loss)
    # n.zero_grad()
    loss.backward()
    for i in n.parameters():
        i.data += -0.01*i.grad
    


Value(data=7.634139461087633)
Value(data=7.490143801407063)
Value(data=7.074326973688169)
Value(data=6.246801795841558)
Value(data=5.0616117101754545)
Value(data=4.347945369445512)
Value(data=4.687407419551814)
Value(data=5.4400456239255135)
Value(data=5.812076079058938)
Value(data=5.390065063535781)
Value(data=5.293513945794615)
Value(data=6.512445270973915)
Value(data=7.537065706740989)
Value(data=7.886683293056949)
Value(data=7.9739797076441405)
Value(data=7.994112554695306)
Value(data=7.998672350144446)
Value(data=7.999700851668242)
Value(data=7.999932611382457)
Value(data=7.999984820927317)


In [3]:
import torch
from engine import Value

def test_sanity_check():

    x = Value(-4.0)
    z = 2 * x + 2 + x
    q = z.relu() + z * x
    h = (z * z).relu()
    y = h + q + q * x
    y.backward()
    xmg, ymg = x, y

    x = torch.Tensor([-4.0]).double()
    x.requires_grad = True
    z = 2 * x + 2 + x
    q = z.relu() + z * x
    h = (z * z).relu()
    y = h + q + q * x
    y.backward()
    xpt, ypt = x, y

    # forward pass went well
    assert ymg.data == ypt.data.item()
    # backward pass went well
    assert xmg.grad == xpt.grad.item()

def test_more_ops():

    a = Value(-4.0)
    b = Value(2.0)
    c = a + b
    d = a * b + b**3
    c += c + 1
    c += 1 + c + (-a)
    d += d * 2 + (b + a).relu()
    d += 3 * d + (b - a).relu()
    e = c - d
    f = e**2
    g = f / 2.0
    g += 10.0 / f
    g.backward()
    amg, bmg, gmg = a, b, g

    a = torch.Tensor([-4.0]).double()
    b = torch.Tensor([2.0]).double()
    a.requires_grad = True
    b.requires_grad = True
    c = a + b
    d = a * b + b**3
    c = c + c + 1
    c = c + 1 + c + (-a)
    d = d + d * 2 + (b + a).relu()
    d = d + 3 * d + (b - a).relu()
    e = c - d
    f = e**2
    g = f / 2.0
    g = g + 10.0 / f
    g.backward()
    apt, bpt, gpt = a, b, g

    tol = 1e-6
    # forward pass went well
    assert abs(gmg.data - gpt.data.item()) < tol
    # backward pass went well
    assert abs(amg.grad - apt.grad.item()) < tol
    assert abs(bmg.grad - bpt.grad.item()) < tol